# HP Tuning

Use the Keras Tuner library to perform hyperparameter tuning on a Classification neural network model. (to classify Fashion MNIST dataset classes)

In [ ]:
import tensorflow as tf
from tensorflow.keras.datasets import fashion_mnist

In [ ]:
# Load dataset
(x_train, y_train), (x_test, y_test) = fashion_mnist.load_data()

# Normalize the pixel values to be between 0 and 1
x_train, x_test = x_train / 255.0, x_test / 255.0

# Reshape data to add a channel dimension (necessary for Conv2D layers)
x_train = x_train.reshape(-1, 28, 28, 1)
x_test = x_test.reshape(-1, 28, 28, 1)

29515/29515 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
26421880/26421880 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step
5148/5148 ━━━━━━━━━━━━━━━━━━━━ 0s 1us/step
4422102/4422102 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step


In [ ]:
from tensorflow.keras import layers
from keras_tuner import HyperModel

In [ ]:
def build_model(hp):
  model = tf.keras.Sequential()

  # Convolutional layers
  model.add(layers.Conv2D(
      filters = hp.Int('filters', min_value = 32, max_value = 128, step = 32),
      kernel_size = hp.Choice('kernel_size', values = [3, 5]),
      activation = 'relu',
      input_shape = (28, 28, 1)
  ))
  model.add(layers.MaxPooling2D(pool_size = (2, 2)))

  # Additional convolutional layers
  model.add(layers.Conv2D(
      filters = hp.Int('filters_2', min_value = 32, max_value = 128, step = 32),
      kernel_size = hp.Choice('kernel_size_2', values = [3, 5]),
      activation = 'relu'
  ))
  model.add(layers.MaxPooling2D(pool_size = (2, 2)))

  model.add(layers.Flatten())

  # Fully connected layer
  model.add(layers.Dense(
      units = hp.Int('units', min_value = 32, max_value = 512, step = 32),
      activation = 'relu'
  ))

  # Output layer
  model.add(layers.Dense(10, activation = 'softmax'))

  # Compile the model
  model.compile(
      optimizer = tf.keras.optimizers.Adam(
          hp.Choice ('learning_rate', values = [1e-2, 1e-1, 1e-4])
      ),
      loss = 'sparse_categorical_crossentropy',
      metrics = ['accuracy']
  )

  return model

## Set up the Keras Tuner

In [ ]:
from keras_tuner import RandomSearch

# Define the tuner
tuner = RandomSearch(
    build_model,
    objective = 'val_accuracy',
    max_trials = 5,
    executions_per_trial = 1,
    directory = 'my_dir',
    project_name = 'fashion_mnist_tuning'
)

/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


## Search for the Best Hyperparameters

In [ ]:
# Search for the best hyperparameters
tuner.search(x_train, y_train, epochs = 10, validation_split = 0.2)

# Get the optimal hyperparameters
best_hps = tuner.get_best_hyperparameters(num_trials =1)[0]

print(f"""
The optimal number of filters in the first Conv2D layer is {best_hps.get('filters')},
the kernel size is {best_hps.get('kernel_size')},
and the learning rate is {best_hps.get('learning_rate')}.
""")

Trial 5 Complete [00h 00m 59s]
val_accuracy: 0.8970833420753479

Best val_accuracy So Far: 0.8970833420753479
Total elapsed time: 00h 05m 44s

The optimal number of filters in the first Conv2D layer is 64,
the kernel size is 3,
and the learning rate is 0.0001.



## Train the best Model

In [ ]:
# Build the model with the best hyperparameters
best_model = tuner.hypermodel.build(best_hps)

# Train the model
best_model.fit(x_train, y_train, epochs = 10, validation_split = 0.2)

# Evaluate on the test data
test_loss, test_acc = best_model.evaluate(x_test, y_test)

Epoch 1/10
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 7s 3ms/step - accuracy: 0.6798 - loss: 0.9607 - val_accuracy: 0.8232 - val_loss: 0.4811
Epoch 2/10
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 9s 3ms/step - accuracy: 0.8400 - loss: 0.4474 - val_accuracy: 0.8582 - val_loss: 0.4047
Epoch 3/10
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 6s 4ms/step - accuracy: 0.8634 - loss: 0.3921 - val_accuracy: 0.8712 - val_loss: 0.3673
Epoch 4/10
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 10s 3ms/step - accuracy: 0.8728 - loss: 0.3579 - val_accuracy: 0.8712 - val_loss: 0.3628
Epoch 5/10
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.8813 - loss: 0.3346 - val_accuracy: 0.8806 - val_loss: 0.3360
Epoch 6/10
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 4s 3ms/step - accuracy: 0.8854 - loss: 0.3160 - val_accuracy: 0.8818 - val_loss: 0.3295
Epoch 7/10
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.8921 - loss: 0.2999 - val_accuracy: 0.8907 - val_loss: 0.3046
Epoch 8/10
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 5s 3ms/step - accuracy: 0.8948 - loss: 0.2859 -

In [ ]:
print(f"Test accuracy: {test_acc}")

Test accuracy: 0.8938999772071838
